In [338]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import numpy as np
import pandas as pd
import wandb

import plotly.express as px

# Pytorch implementation

In [339]:
use_wandb = False

In [340]:
config = {
    "lr": 1e-3,
    "dataset": "Titanic",
    "epochs": 100,
    "batch_size": 64,
    "hidden_layers": [64, 32],
    "dropouts": [0.4],
    "scheduler": "exponential",
    "activations": "ReLU",
    "loss": "binary cross-entropy",
    "optimizer": "Adam"
}

In [341]:
if use_wandb:
    wandb.login()
    wandb.init(project="assignment-1", name="pytorch-64-32", reinit=True, config=config)
    # wandb.config = config

In [342]:
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


## Loading the dataset

In [343]:
class TitanicDataset(Dataset):
    def __init__(self, data_dir):
        self.labels = pd.read_csv(os.path.join(data_dir, 'labels.csv')).to_numpy(dtype='float32')
        self.data = pd.read_csv(os.path.join(data_dir, 'data.csv')).to_numpy(dtype='float32')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = torch.from_numpy(self.data[idx])
        label = torch.from_numpy(self.labels[idx])

        return data, label

In [344]:
train_data = TitanicDataset(data_dir='data/train/')
val_data = TitanicDataset(data_dir='data/val/')
# test_data = TitanicDataset(data_dir='data/test/')

In [345]:
features, label = train_data.__getitem__(idx=0)
print(features, label)

tensor([2.0000, 1.0000, 0.3324, 1.0000, 1.0000, 0.5848]) tensor([1.])


In [346]:
train_dataloader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=False)
# test_dataloader = DataLoader(test_data, batch_size=config['batch_size'], shuffle=False)

In [347]:
for batch, (x, y) in enumerate(train_dataloader):
    print(batch, x.shape, y.shape)
    break

0 torch.Size([64, 6]) torch.Size([64, 1])


## Defining the model

In [348]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(6, config['hidden_layers'][0]),
            # nn.Dropout(config['dropouts'][0]),
            nn.ReLU(),
            nn.Linear(config['hidden_layers'][0], config['hidden_layers'][1]),
            nn.Dropout(config['dropouts'][0]),
            nn.ReLU(),
            nn.Linear(config['hidden_layers'][1], 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.linear(x)
        return out

In [349]:
model = NeuralNetwork().to(device)
print(model)

# if use_wandb: 
#     wandb.config['model'] = model.__dict__['_modules']['linear']

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=1, bias=True)
    (6): Sigmoid()
  )
)


## Training

In [350]:
loss_fn = nn.BCELoss()
if config['optimizer'] == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], betas=[0.9, 0.99])
elif config['optimizer'] == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'])
elif config['optimizer'] == 'RMSprop':
    optimizer = torch.optim.RMSprop(model.parameters(), lr=config['lr'])

In [351]:
# if use_wandb:
#     wandb.watch(model, loss_fn, log='all', log_freq=10)

In [352]:
def calculate_acc(y_pred, y) -> float:
    y_pred = (y_pred>0.5)
    
    return ((y == y_pred).sum().item())/len(y)

In [353]:
def train(dataloader, model, loss_fn, optimizer, scheduler, epoch):
    model.train()
    avg_loss, avg_acc = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        # Get prediction and compute loss
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        avg_loss += loss.item()

        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_acc += calculate_acc(y_pred, y)

    # Change lr based on the provided scheduler
    if epoch > 9:
        scheduler.step()
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
        
    return (avg_loss, avg_acc)

In [354]:
def val(dataloader, model, loss_fn, epoch):    
    model.eval()
    avg_loss, avg_acc = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            avg_loss += loss_fn(y_pred, y).item()
            avg_acc += calculate_acc(y_pred, y)
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
    
    return (avg_loss, avg_acc)

In [355]:
epochs = config['epochs']
train_history = {'loss': [], 'acc': []}
val_history = {'loss': [], 'acc': []}
best_epoch = {}

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
for epoch in range(epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer, scheduler, epoch)
    val_loss, val_acc = val(val_dataloader, model, loss_fn, epoch)

    train_history['loss'].append(train_loss)
    train_history['acc'].append(train_acc)

    val_history['loss'].append(val_loss)
    val_history['acc'].append(val_acc)
    
    if (epoch % 10 == 0):
        print(f"Epoch {epoch}")
        print(f'loss: {train_loss:>5f} acc: {train_acc:>5f}')
        print(f'val loss: {val_loss:>5f} val acc: {val_acc:>5f}')
        print(f'lr: {optimizer.param_groups[0]["lr"]:>5f}')
        print('-------------------------------')

    if use_wandb:
        wandb.log({'epoch': epoch, 'loss': train_loss, 'accuracy': train_acc, 'val_loss':val_loss, 'val_accuracy': val_acc, 'lr': optimizer.param_groups[0]["lr"]})

Epoch 0
loss: 0.684159 acc: 0.575229
val loss: 0.655983 val acc: 0.743403
lr: 0.001000
-------------------------------
Epoch 10
loss: 0.476851 acc: 0.764482
val loss: 0.463761 val acc: 0.799653
lr: 0.000990
-------------------------------
Epoch 20
loss: 0.420195 acc: 0.814367
val loss: 0.431125 val acc: 0.824653
lr: 0.000895
-------------------------------
Epoch 30
loss: 0.408879 acc: 0.810175
val loss: 0.421998 val acc: 0.824653
lr: 0.000810
-------------------------------
Epoch 40
loss: 0.410365 acc: 0.829421
val loss: 0.417744 val acc: 0.830903
lr: 0.000732
-------------------------------
Epoch 50
loss: 0.394774 acc: 0.830488
val loss: 0.415794 val acc: 0.834028
lr: 0.000662
-------------------------------
Epoch 60
loss: 0.381828 acc: 0.848056
val loss: 0.412981 val acc: 0.834028
lr: 0.000599
-------------------------------
Epoch 70
loss: 0.380118 acc: 0.843369
val loss: 0.414871 val acc: 0.837153
lr: 0.000542
-------------------------------
Epoch 80
loss: 0.381833 acc: 0.847980
val

In [356]:
print(f'Results after {config["epochs"]} epochs for a model with:\n{config["optimizer"]} optimizer, {config["lr"]} lr, {config["batch_size"]} batch size, {config["loss"]} and with the following architecture:')
print(model)

if use_wandb:
    wandb.finish()

Results after 100 epochs for a model with:
Adam optimizer, 0.001 lr, 64 batch size, binary cross-entropy and with the following architecture:
NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=1, bias=True)
    (6): Sigmoid()
  )
)


In [357]:
fig = px.line({'loss': train_history['loss'], 'val_loss': val_history['loss']})
fig.show()

In [358]:
fig = px.line({'acc': train_history['acc'], 'val_acc': val_history['acc']})
fig.show()